# Run WhisperAI using Google Colab's GPU

To enable GPU in this notebook, select Runtime -> Change runtime type in the Menu bar. Under Hardware Accelerator, select GPU.

Then, go to the Global variables [cell](#scrollTo=nEyzcieyHhav&line=3&uniqifier=1) and update it with your ngrok authentication token.

To run WhisperAI, select Runtime -> Run all. Go to this [cell](#scrollTo=hCrK1VjzOpJk&line=8&uniqifier=1) and read the instructions on how to update your `.env` file to use WhisperAI running in this notebook.

## Pip install dependencies

In [ ]:
!pip install flask-ngrok2 -q
!pip install git+https://github.com/openai/whisper.git -q

## Imports

In [ ]:
from flask_ngrok2 import run_with_ngrok
from flask import Flask, request 
import json
import whisper

## Global variables

Get your ngrok authentication token by signing up [here](https://dashboard.ngrok.com/signup). After signing up, copy your auth token [here](https://dashboard.ngrok.com/get-started/your-authtoken) and update the NGROK_AUTH_TOKEN variable with your token.

In [ ]:
# Uncomment the line below and paste your ngrok auth token
NGROK_AUTH_TOKEN = '2M8j0MFxiA90NwVjwSxDIb3vZif_2SkUFXh69AYsab1DN5DVv'
TRANSLATE_FILENAME = 'translate.wav'
TRANSCRIBE_FILENAME = 'transcribe.wav'
WHISPER_MODEL = 'small'

## Load WhisperAI model

In [ ]:
model = whisper.load_model(WHISPER_MODEL)

## Run WhisperAI and host it using ngrok

After running, copy the public url provided by ngrok and update WHISPER_BASE_URL in your `.env` file. Below is an example output that you will see. 

```
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://9d55-35-192-10-95.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
```

DO NOT use this url, use the URL provided by the actual output from running the cell below. In this example, you will update your WHISPER_BASE_URL variable with:

```
WHISPER_BASE_URL=http://9d55-35-192-10-95.ngrok.io
```

This url will change every time you rerun this cell, so remember to update your `.env` file when that happens.

In [ ]:
app = Flask(__name__)
run_with_ngrok(app, auth_token=NGROK_AUTH_TOKEN)


@app.route('/', methods=['GET'])
def test():
    response = {'status':'OK','message':'Test'}
    return json.dumps(response)


@app.route('/asr', methods=['POST'])
def asr():
    task = request.args.get('task')
    language = request.args.get('language')
    audio_data = request.files['audio_file']

    if task == 'transcribe':
      audio_data.save(TRANSCRIBE_FILENAME)
      result = model.transcribe(TRANSCRIBE_FILENAME)
      return json.dumps(result)

    elif task == 'translate':
      audio_data.save(TRANSLATE_FILENAME)
      result = model.transcribe(TRANSLATE_FILENAME, language='ja', task='translate')
      return json.dumps(result)
    
    else:
      return 'Record not found', 400


if __name__ == '__main__':
   app.run()

## Leave the above cell running and the tab open

This is to ensure the runtime does not disconnect and shut down the server. 

When you're done using WhisperAI remember to disconnect the runtime.